In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
import time
import random
import os

In [6]:
def generate_date_list(start, end):
    date_list = []
    curr = end
    while curr >= start:
        date_list.append(curr.strftime("%Y.%m.%d"))
        curr -= timedelta(days=1)
    return date_list

In [7]:
def extract_urls(json_data):
    urls = []
    for item in json_data.get('collection', []):
        html_str = item.get('html', '')
        soup = BeautifulSoup(html_str, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            if 'n.news.naver.com' in link['href']:
                urls.append(link['href'])
    unique_urls = list(set(urls))
    return unique_urls

In [8]:
def get_urls_bydate(date, office_id):
    collected_urls = []
    base_url = "https://s.search.naver.com/p/newssearch/3/api/tab/more"
    headers = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://search.naver.com/'}
    for start in range(1, 2000, 10):
        params = {
        'abt': 'null',
        'de': date,
        'ds': date,
        'field': '0',
        'is_dts': '0',
        'is_sug_officeid': '0',
        'mynews': '1',
        'news_office_checked': office_id,
        'nqx_theme': '{"theme":{"sub":[{"name":"finance"}]}}',
        'nso': f'so:r,p:from{date.replace(".","")}to{date.replace(".","")},a:all',
        'office_category': '0',
        'office_section_code': '3',
        'office_type': '1',
        'pd': '3',
        'photo': '0',
        'query': '금리',
        'rev': '0',
        'service_area': '0',
        'sm': 'tab_smr',
        'sort': '2',
        'spq': '0',
        'ssc': 'tab.news.all',
        'start': start
        }
        try:
            res = requests.get(base_url, headers=headers, params=params, timeout=10)
            if res.status_code != 200 or not res.text.strip():
                print(f"[{date}] 수집 종료")
                break
            data = res.json()
            page_urls = extract_urls(data)
            
            if not page_urls:
                break
            
            collected_urls.extend(page_urls)
            
            time.sleep(random.uniform(0.3, 0.6))
            
        except Exception as e:
            print(f"[{date}] 에러 발생: {e}")
            raise e
            
    return list(set(collected_urls))

In [9]:
offices = {"매일경제": "1009", "한국경제": "1015", "머니투데이": "1008"}
keyword = "금리"
s_dt = datetime(2012, 1, 1) 
e_dt = datetime(2025, 12, 30)

In [ ]:
for office in offices:
    office_id = offices[office]
    file_name = f'news_urls_{office}.csv'

    seen_urls = set()
    if os.path.exists(file_name):
        existing_df = pd.read_csv(file_name)
        seen_urls = set(existing_df['url'].tolist())
        print(f"✅ {office}: 기존 파일에서 {len(seen_urls)}개의 주소를 확인했습니다.")
     
    target_dates = generate_date_list(s_dt, e_dt)
    for i, date in enumerate(target_dates):
        print(f"\n{office} [진행도: {i+1}/{len(target_dates)}] {date} 수집 중...")
        urls = get_urls_bydate(date, office_id)

    if urls:
        new_urls = [u for u in urls if u not in seen_urls]
        if new_urls:
            df = pd.DataFrame(new_urls, columns=['url'])
            df['date'] = date
            if not os.path.exists(file_name):
                df.to_csv(file_name, index=False, encoding='utf-8-sig')
            else:
                df.to_csv(file_name, index=False, encoding='utf-8-sig', mode='a', header=False)
                print(f"{office} {date} 저장 완료({len(new_urls)}개 추가됨)")
                seen_urls.update(new_urls)
        else:
            print(f"{office} {date}: 모두 중복되어 건너뜁니다.")
    else:
        print(f"{office} {date}: 검색 결과 없음.")

    print(f"\n✨ {office} 수집 완료")

✅ 매일경제: 기존 파일에서 1530개의 주소를 확인했습니다.

매일경제 [진행도: 1/5113] 2025.12.30 수집 중...

매일경제 [진행도: 2/5113] 2025.12.29 수집 중...

매일경제 [진행도: 3/5113] 2025.12.28 수집 중...

매일경제 [진행도: 4/5113] 2025.12.27 수집 중...

매일경제 [진행도: 5/5113] 2025.12.26 수집 중...

매일경제 [진행도: 6/5113] 2025.12.25 수집 중...

매일경제 [진행도: 7/5113] 2025.12.24 수집 중...

매일경제 [진행도: 8/5113] 2025.12.23 수집 중...

매일경제 [진행도: 9/5113] 2025.12.22 수집 중...

매일경제 [진행도: 10/5113] 2025.12.21 수집 중...

매일경제 [진행도: 11/5113] 2025.12.20 수집 중...

매일경제 [진행도: 12/5113] 2025.12.19 수집 중...

매일경제 [진행도: 13/5113] 2025.12.18 수집 중...

매일경제 [진행도: 14/5113] 2025.12.17 수집 중...

매일경제 [진행도: 15/5113] 2025.12.16 수집 중...

매일경제 [진행도: 16/5113] 2025.12.15 수집 중...

매일경제 [진행도: 17/5113] 2025.12.14 수집 중...

매일경제 [진행도: 18/5113] 2025.12.13 수집 중...

매일경제 [진행도: 19/5113] 2025.12.12 수집 중...

매일경제 [진행도: 20/5113] 2025.12.11 수집 중...

매일경제 [진행도: 21/5113] 2025.12.10 수집 중...

매일경제 [진행도: 22/5113] 2025.12.09 수집 중...

매일경제 [진행도: 23/5113] 2025.12.08 수집 중...

매일경제 [진행도: 24/5113] 2025.12.07 수집 중...

매일경제 [진행도: 25

In [ ]:
# import threading

# csv_lock = threading.Lock()

# def collect_urls(office, s_dt, e_dt, max_workers=2):
#     office_id = offices[office]
#     file_name = f'news_urls_{office}.csv'

#     seen_urls = set()
#     if os.path.exists(file_name):
#         existing_df = pd.read_csv(file_name)
#         seen_urls = set(existing_df['url'].tolist())
#         print(f"✅ {office}: 기존 파일에서 {len(seen_urls)}개의 주소를 확인했습니다.")
    
#     target_dates = generate_date_list(s_dt, e_dt)

#     with ThreadPoolExecutor(max_workers=max_workers) as executor:
#         future_to_date = {executor.submit(get_urls_bydate, date, office_id): date for date in target_dates}
#         for future in as_completed(future_to_date):
#             date = future_to_date[future]
#             try:
#                 urls = future.result()
#                 if urls:
#                     new_urls = [u for u in urls if u not in seen_urls]
#                     if new_urls:
#                         df = pd.DataFrame(new_urls, columns=['url'])
#                         df['date'] = date

#                         with csv_lock:
#                             if not os.path.exists(file_name):
#                                 df.to_csv(file_name, index=False, encoding='utf-8-sig')
#                             else:
#                                 df.to_csv(file_name, index=False, encoding='utf-8-sig', mode='a', header=False)
#                             print(f"{office} {date} 저장 완료({len(new_urls)}개 추가됨)")
#                             seen_urls.update(new_urls)
#                     else:
#                         print(f"{office} {date}: 모두 중복되어 건너뜁니다.")
#                 else:
#                     print(f"{office} {date}: 검색 결과 없음.")
#             except Exception as e:
#                 print(f"❌ {date} 에러: {e}")
#     print(f"\n✨ {office} 수집 완료")

In [ ]:
# for office in offices:
#     collect_urls(office, s_dt, e_dt, max_workers=2)

✅ 매일경제: 기존 파일에서 1520개의 주소를 확인했습니다.
[2025.12.29] 수집 종료
매일경제 2025.12.29: 검색 결과 없음.
[2025.12.30] 수집 종료
매일경제 2025.12.30: 검색 결과 없음.
[2025.12.28] 수집 종료
매일경제 2025.12.28: 검색 결과 없음.
[2025.12.27] 수집 종료
매일경제 2025.12.27: 검색 결과 없음.
[2025.12.26] 수집 종료
매일경제 2025.12.26: 검색 결과 없음.
[2025.12.24] 수집 종료
매일경제 2025.12.24: 검색 결과 없음.
[2025.12.25] 수집 종료
매일경제 2025.12.25: 검색 결과 없음.
[2025.12.23] 수집 종료
매일경제 2025.12.23: 검색 결과 없음.
[2025.12.22] 수집 종료
매일경제 2025.12.22: 검색 결과 없음.
[2025.12.21] 수집 종료
매일경제 2025.12.21: 검색 결과 없음.
[2025.12.20] 수집 종료
매일경제 2025.12.20: 검색 결과 없음.
[2025.12.19] 수집 종료
매일경제 2025.12.19: 검색 결과 없음.
[2025.12.18] 수집 종료
매일경제 2025.12.18: 검색 결과 없음.
[2025.12.17] 수집 종료
매일경제 2025.12.17: 검색 결과 없음.
[2025.12.16] 수집 종료
매일경제 2025.12.16: 검색 결과 없음.
[2025.12.15] 수집 종료
매일경제 2025.12.15: 검색 결과 없음.
[2025.12.14] 수집 종료
매일경제 2025.12.14: 검색 결과 없음.
[2025.12.13] 수집 종료
매일경제 2025.12.13: 검색 결과 없음.
[2025.12.12] 수집 종료
매일경제 2025.12.12: 검색 결과 없음.
[2025.12.11] 수집 종료
매일경제 2025.12.11: 검색 결과 없음.
[2025.12.10] 수집 종료
매일경제 2025.12.10: 검색 결과 없음.

In [ ]:
# 뉴스 본문 수집
def news_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Referer': 'https://news.naver.com/'
    }
    try:
        res = requests.get(url, headers=headers, timeout=15)
        if res.status_code != 200:
            return None
        soup = BeautifulSoup(res.text, 'html.parser')
        
        title = soup.select_one("h2#title_area")
        title = title.get_text(strip=True) if title else "제목 없음"
        
        date_tag = soup.select_one("span.media_end_head_info_datestamp_time")
        if date_tag and date_tag.has_attr('data-date-time'):
            date_str = date_tag['data-date-time']
        else:
            date_str = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
        content = soup.select_one("#newsct_article") # 또는 "#dic_area"
        
        if content:
            content = content.get_text(" ", strip=True)
        else:
            content = "본문 없음"
        return {'title': title, 'date': date_str, 'content': content, 'url': url}

    except Exception as e:
        print(f"\n[{url}] 수집 중 에러 발생: {e}") 
        raise e

In [ ]:
for office in offices:
    input_file = f"news_urls_{office}.csv"
    output_file = f"news_contents_{office}.csv"
    max_workers = 4

    df_input = pd.read_csv(input_file)
    full_urls = set(df_input['url'].tolist())

    already_done_urls = set()
    if os.path.exists(output_file):
        df_existing = pd.read_csv(output_file)
        already_done_urls = set(df_existing['url'].tolist())
        print(f"기존 파일에서 {len(already_done_urls)}개의 수집 완료 데이터를 찾았습니다.")

    urls_to_crawl = [u for u in full_urls if u not in already_done_urls]
    print(f"수집해야 할 기사는 {len(urls_to_crawl)}개입니다.")


    results = []
    # 병렬 처리
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(news_content, url): url for url in urls_to_crawl}
        for i, future in enumerate(as_completed(future_to_url)):
            try:
                data = future.result()
                if data:
                    results.append(data)
            except Exception as e:
                print(f"❌ 에러 발생: {e}")
            # 100개마다 중간 저장
            if (i + 1) % 100 == 0:
                pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig', 
                                            mode='a', header=not os.path.exists(output_file))
                results = [] # 저장 후 메모리 비우기
                print(f"✅{office} {i + 1}개 완료 및 중간 저장됨...")
                
                # 차단 방지
                time.sleep(1)

    # 남은 데이터 저장
    if results:
        pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig', 
                                    mode='a', header=not os.path.exists(output_file))

    print(f"\n🎉 모두 수집 완료! '{output_file}'을 확인하세요.")

기존 파일에서 90개의 수집 완료 데이터를 찾았습니다.
수집해야 할 기사는 0개입니다.

🎉 모두 수집 완료! 'news_contents_매일경제.csv'을 확인하세요.
기존 파일에서 142개의 수집 완료 데이터를 찾았습니다.
수집해야 할 기사는 0개입니다.

🎉 모두 수집 완료! 'news_contents_한국경제.csv'을 확인하세요.
기존 파일에서 330개의 수집 완료 데이터를 찾았습니다.
수집해야 할 기사는 0개입니다.

🎉 모두 수집 완료! 'news_contents_머니투데이.csv'을 확인하세요.
